In [1]:
import numpy as np 
from keras.datasets import mnist 
from keras.layers import Dense,Flatten,Reshape 
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential 
from keras.optimizers import Adam 
import matplotlib.pyplot as plt 


Using TensorFlow backend.


In [3]:
#Input image dimensions 
imgRows = 28; imgCols = 28; imgChannels= 1
imgShape = (imgRows,imgCols,imgChannels)
#The input to the generator is the noise vector
zDim = 100

In [15]:
#Build the generator
def buildGenerator(imgShape,zDim):
    model = Sequential()
    model.add(Dense(128,input_dim = zDim))
    model.add(LeakyReLU(alpha = 0.01))
    #Output layer with image dimensions
    model.add(Dense(28*28*1,activation = 'tanh'))
    #Reshape according to the image dimensions
    model.add(Reshape(imgShape))
    return model 

In [16]:
def buildDiscriminator(imgShape):
    model = Sequential()
    #Flatten the image 
    model.add(Flatten(input_shape = imgShape))
    #FC layer 
    model.add(Dense(128))
    #Leaky Relu activation 
    model.add(LeakyReLU(alpha = 0.01))
    #Output with Sigmoid activation 
    model.add(Dense(1,activation = 'sigmoid'))
    return model 

In [17]:
#Build the GAN model 
def buildGAN(generator,discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [18]:
discriminator = buildDiscriminator(imgShape)
discriminator.compile(loss = 'binary_crossentropy',optimizer = Adam(),metrics = ['accuracy'])
#Build the generator 
generator = buildGenerator(imgShape,zDim)

#Keep Discriminator's parameters for Generator's training 
discriminator.trainable = False 

#Build the GAN 
GAN = buildGAN(generator,discriminator)
GAN.compile(loss = 'binary_crossentropy',optimizer = Adam())


In [23]:
def sample_images(generator,image_grid_rows = 4,image_grid_cols = 4):
    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_cols, zDim))

    # Generate images from random noise
    genImgs = generator.predict(z)

    # Rescale image pixel values to [0, 1]
    genImgs = 0.5 * genImgs + 0.5

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_cols,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_cols):
            # Output a grid of images
            axs[i, j].imshow(genImgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

In [ ]:
#Training 
losses = []
accuracies = []
iterationCheckpoints = []
def train(iterations,batch_size,sample_interval):
    (x_train,_),(_,_) = mnist.load_data()
    x_train = x_train/127.0 - 1.0
    x_train = np.expand_dims(x_train,axis = 3)
    
    #Label all real images as 1 and all fakes ones as zero 
    real = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    for iteration in range(iterations):
        #Train the discriminator 
        #Random batch of real images
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]
        
        #Generate a batch of fake images 
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)
        
        #Actual training
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #Train the generator 
        #Generate fake images 
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)
        
        #Actual training 
        g_loss = GAN.train_on_batch(z, real)
        
        if((iteration+1) % sample_interval) == 0:
            #Save the losses and accuracies 
            losses.append((d_loss,g_loss))
            accuracies.append(100.0*accuracy)
            iterationCheckpoints.append(iteration+1)
            
            #Output a sample of the generated image 
            sample_images(generator)
        
    

In [ ]:
#Train the GAN and inspect the output 
#Hyperparameters 
iterations = 20000
batch_size = 128
sample_interval = 1000

#train the GAN 
train(iterations, batch_size, sample_interval)


In [ ]:
# Plot training losses for Discriminator and Generator
plt.figure(figsize=(15, 5))
plt.plot(iterationCheckpoints, losses.T[0], label="Discriminator loss")
plt.plot(iterationCheckpoints, losses.T[1], label="Generator loss")

plt.xticks(iterationCheckpoints, rotation=90)

plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
accuracies = np.array(accuracies)

# Plot Discriminator accuracy
plt.figure(figsize=(15, 5))
plt.plot(iterationCheckpoints, accuracies, label="Discriminator accuracy")

plt.xticks(iterationCheckpoints, rotation=90)
plt.yticks(range(0, 100, 5))

plt.title("Discriminator Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()